# Package Output Data
2023-12-27 ZD  

Workspace for code to pacakge and prepare gathered data for input into INS as formatted TSVs. Any useful scripts will be moved from here to `modules/package_output_data.py` for production use. 

In [1]:
# Method to import from parent directory
import os
import sys
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(root_dir)
import config
import re
from datetime import datetime
import unicodedata

import pandas as pd
from tqdm import tqdm
import numpy as np

# Get all existing publication functions
import modules.package_output_data as pack



---TIMESTAMP OVERRIDE IN USE---
---Performing action using 2023-12-22 instead of current timestamp.---
---Change the OVERRIDE_DATE in config.py to None to restore default behavior.---




In [4]:
df = pd.read_csv('../'+config.PROJECTS_INTERMED_PATH)
df

,project_id,queried_project_id,application_id,fiscal_year,project_title,abstract_text,keywords,principal_investigators,program_officers,award_amount,...,award_notice_date,project_start_date,project_end_date,opportunity_number,api_source_search,org_name,org_city,org_state,org_country,program.program_id
0,1R01CA239701-01A1,R01CA239701,9995648,2020,Admixture analysis of acute lymphoblastic leuk...,Abstract Children with substantial African anc...,Accounting;Acute Lymphocytic Leukemia;Admixtur...,Michael E Scheurer; Logan G. Spector,Danielle L Daee,1898254,...,2020-07-23T12:07:00Z,2020-08-01T12:08:00Z,2023-07-31T12:07:00Z,PA-19-056,award_R01CA239701,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...
1,3R01CA239701-01A1S1,R01CA239701,10307680,2021,Admixture analysis of acute lymphoblastic leuk...,This attachment is not required for the divers...,Acute Lymphocytic Leukemia;Admixture;African A...,Michael E Scheurer; Logan G. Spector,Danielle L Daee,11554,...,2021-09-15T12:09:00Z,2021-08-01T12:08:00Z,2022-07-31T12:07:00Z,PA-21-071,award_R01CA239701,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...
2,3R01CA239701-01A1S3,R01CA239701,10626271,2022,Admixture analysis of acute lymphoblastic leuk...,Abstract This application is being submitted i...,Acute Lymphocytic Leukemia;Administrative Supp...,Joseph Lubega; Michael E Scheurer; Logan G. Sp...,Danielle L Daee,197648,...,2022-09-19T12:09:00Z,2020-08-01T12:08:00Z,2023-07-31T12:07:00Z,PA-20-272,award_R01CA239701,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...
3,4R01CA239701-02,R01CA239701,10902170,2023,Admixture analysis of acute lymphoblastic leuk...,Modified Project Summary/Abstract Section Abst...,Acute Lymphocytic Leukemia;Admixture;African;A...,Michael E Scheurer; Logan G. Spector,Danielle L Daee,635018,...,2023-09-15T12:09:00Z,2020-08-01T12:08:00Z,2025-07-31T12:07:00Z,PA-19-056,award_R01CA239701,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...
4,1R21CA209848-01,R21CA209848,9185145,2016,Algorithms for Literature-Guided Multi-Platfor...,Project Abstract The development of accurate a...,Accounting;Address;Algorithms;Bayesian Modelin...,Dongjun Chung; Linda E Kelemen,David J Miller,195098,...,2016-07-27T12:07:00Z,2016-08-01T12:08:00Z,2018-07-31T12:07:00Z,PAR-15-334,nofo_PAR-15-334,MEDICAL UNIVERSITY OF SOUTH CAROLINA,CHARLESTON,SC,UNITED STATES,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,5P50CA196530-04,P50CA196530,9529550,2018,Yale SPORE in Lung Cancer (YSILC): The Biology...,DESCRIPTION (provided by applicant): The Yale ...,American;Animal Disease Models;Animal Model;Be...,Roy S Herbst,Peter Ujhazy,2507598,...,2018-08-20T12:08:00Z,2015-08-26T12:08:00Z,2020-07-31T12:07:00Z,PAR-14-031,award_P50CA196530,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES,YaleSPOREinLungCancerYSILCTheBiologyandPersona...
5996,5P50CA196530-05,P50CA196530,9767058,2019,Yale SPORE in Lung Cancer (YSILC): The Biology...,DESCRIPTION (provided by applicant): The Yale ...,American;Animal Disease Models;Animal Model;Be...,Roy S Herbst,Peter Ujhazy,2134771,...,2019-08-21T12:08:00Z,2015-08-26T12:08:00Z,2020-07-31T12:07:00Z,PAR-14-031,award_P50CA196530,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES,YaleSPOREinLungCancerYSILCTheBiologyandPersona...
5997,5P50CA196530-07,P50CA196530,10203850,2021,Yale SPORE in Lung Cancer (YSILC): The Biology...,YALE SPORE IN LUNG CANCER (YSILC) OVERALL SUMM...,Address;Adoption;Animal Disease Models;Area;Aw...,Roy S Herbst,Peter Ujhazy,2003668,...,2021-09-01T12:09:00Z,2015-08-26T12:08:00Z,2025-07-31T12:07:00Z,PAR-18-313,award_P50CA196530,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES,YaleSPOREinLungCancerYSILCTheBiologyandPersona...
5998,5P50CA196530-08,P50CA196530,10479786,2022,Yale SPORE in Lung 

In [5]:
df.groupby('project_id').size().reset_index().sort_values(by=0, ascending=False)

,project_id,0
2604,3P50CA171963-10S1,2
5506,5U24CA224020-03,2
2193,2U24CA224319-02,2
2192,2U24CA224309-02,2
2191,2U24CA224285-02,2
...,...,...
1984,2R01CA053840-29,1
1983,2R01CA047296-34,1
1982,2R01CA031845-41A1,1
1981,2P50CA221747-06,1


In [6]:
df[df['project_id'] == '3P50CA171963-10S1']

,project_id,queried_project_id,application_id,fiscal_year,project_title,abstract_text,keywords,principal_investigators,program_officers,award_amount,...,award_notice_date,project_start_date,project_end_date,opportunity_number,api_source_search,org_name,org_city,org_state,org_country,program.program_id
2285,3P50CA171963-10S1,P50CA171963,10912197,2023,Specialized Program Of Research Excellence (SP...,ABSTRACT The Washington University SPORE in Le...,AML/MDS;Accreditation;Acute;Acute Lymphocytic ...,Daniel C Link,Igor A Kuzmin,918606,...,2023-09-20T12:09:00Z,2013-09-03T12:09:00Z,2024-06-30T12:06:00Z,PA-20-272,nofo_PA20-272,WASHINGTON UNIVERSITY,SAINT LOUIS,MO,UNITED STATES,AllofUs
5679,3P50CA171963-10S1,P50CA171963,10912197,2023,Specialized Program Of Research Excellence (SP...,ABSTRACT The Washington University SPORE in Le...,AML/MDS;Accreditation;Acute;Acute Lymphocytic ...,Daniel C Link,Igor A Kuzmin,918606,...,2023-09-20T12:09:00Z,2013-09-03T12:09:00Z,2024-06-30T12:06:00Z,PA-20-272,award_P50CA171963,WASHINGTON UNIVERSITY,SAINT LOUIS,MO,UNITED STATES,SpecializedProgramOfResearchExcellenceSPOREinL...


In [21]:
df2 = df.drop(columns=['api_source_search','program.program_id'])
df2

,project_id,queried_project_id,application_id,fiscal_year,project_title,abstract_text,keywords,principal_investigators,program_officers,award_amount,nci_funded_amount,award_notice_date,project_start_date,project_end_date,opportunity_number,org_name,org_city,org_state,org_country
0,1R01CA239701-01A1,R01CA239701,9995648,2020,Admixture analysis of acute lymphoblastic leuk...,Abstract Children with substantial African anc...,Accounting;Acute Lymphocytic Leukemia;Admixtur...,Michael E Scheurer; Logan G. Spector,Danielle L Daee,1898254,1898254,2020-07-23T12:07:00Z,2020-08-01T12:08:00Z,2023-07-31T12:07:00Z,PA-19-056,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES
1,3R01CA239701-01A1S1,R01CA239701,10307680,2021,Admixture analysis of acute lymphoblastic leuk...,This attachment is not required for the divers...,Acute Lymphocytic Leukemia;Admixture;African A...,Michael E Scheurer; Logan G. Spector,Danielle L Daee,11554,11554,2021-09-15T12:09:00Z,2021-08-01T12:08:00Z,2022-07-31T12:07:00Z,PA-21-071,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES
2,3R01CA239701-01A1S3,R01CA239701,10626271,2022,Admixture analysis of acute lymphoblastic leuk...,Abstract This application is being submitted i...,Acute Lymphocytic Leukemia;Administrative Supp...,Joseph Lubega; Michael E Scheurer; Logan G. Sp...,Danielle L Daee,197648,197648,2022-09-19T12:09:00Z,2020-08-01T12:08:00Z,2023-07-31T12:07:00Z,PA-20-272,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES
3,4R01CA239701-02,R01CA239701,10902170,2023,Admixture analysis of acute lymphoblastic leuk...,Modified Project Summary/Abstract Section Abst...,Acute Lymphocytic Leukemia;Admixture;African;A...,Michael E Scheurer; Logan G. Spector,Danielle L Daee,635018,635018,2023-09-15T12:09:00Z,2020-08-01T12:08:00Z,2025-07-31T12:07:00Z,PA-19-056,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES
4,1R21CA209848-01,R21CA209848,9185145,2016,Algorithms for Literature-Guided Multi-Platfor...,Project Abstract The development of accurate a...,Accounting;Address;Algorithms;Bayesian Modelin...,Dongjun Chung; Linda E Kelemen,David J Miller,195098,195098,2016-07-27T12:07:00Z,2016-08-01T12:08:00Z,2018-07-31T12:07:00Z,PAR-15-334,MEDICAL UNIVERSITY OF SOUTH CAROLINA,CHARLESTON,SC,UNITED STATES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,5P50CA196530-04,P50CA196530,9529550,2018,Yale SPORE in Lung Cancer (YSILC): The Biology...,DESCRIPTION (provided by applicant): The Yale ...,American;Animal Disease Models;Animal Model;Be...,Roy S Herbst,Peter Ujhazy,2507598,2507598,2018-08-20T12:08:00Z,2015-08-26T12:08:00Z,2020-07-31T12:07:00Z,PAR-14-031,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES
5996,5P50CA196530-05,P50CA196530,9767058,2019,Yale SPORE in Lung Cancer (YSILC): The Biology...,DESCRIPTION (provided by applicant): The Yale ...,American;Animal Disease Models;Animal Model;Be...,Roy S Herbst,Peter Ujhazy,2134771,2134771,2019-08-21T12:08:00Z,2015-08-26T12:08:00Z,2020-07-31T12:07:00Z,PAR-14-031,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES
5997,5P50CA196530-07,P50CA196530,10203850,2021,Yale SPORE in Lung Cancer (YSILC): The Biology...,YALE SPORE IN LUNG CANCER (YSILC) OVERALL SUMM...,Address;Adoption;Animal Disease Models;Area;Aw...,Roy S Herbst,Peter Ujhazy,2003668,2003668,2021-09-01T12:09:00Z,2015-08-26T12:08:00Z,2025-07-31T12:07:00Z,PAR-18-313,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES
5998,5P50CA196530-08,P50CA196530,10479786,2022,Yale SPORE in Lung Cancer (YSILC): The Biology...,YALE SPORE IN LUNG CANCER (YSILC) OVERALL SUMM...,Address;Adoption;Animal Disease Models;Area;Aw...,Roy S Herbst; Katerina Abigail Politi,Peter Ujhazy,2046999,2046999,2022-08-23T12:08:00Z,2015-08-26T12:08:00Z,2025-07-31T12:07:00Z,PAR-18-313,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES


In [12]:
df2[df2.duplicated(subset=['project_id'], keep=False)].sort_values(by='project_id')

,project_id,queried_project_id,application_id,fiscal_year,project_title,abstract_text,keywords,principal_investigators,program_officers,award_amount,nci_funded_amount,award_notice_date,project_start_date,project_end_date,opportunity_number,org_name,org_city,org_state,org_country
5150,1U01CA224145-01,U01CA224145,9449550,2017,Interrupting Cellular Crosstalk in the Immunos...,Abstract Immune therapy has shown great promis...,Ablation;Adenocarcinoma Cell;Affect;Animal Mod...,Howard C Crawford; Marina Pasca Di Magliano,Jeffrey Hildesheim,1497616,1497616,2017-09-25T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ANN ARBOR,MI,UNITED STATES
4080,1U01CA224145-01,U01CA224145,9449550,2017,Interrupting Cellular Crosstalk in the Immunos...,Abstract Immune therapy has shown great promis...,Ablation;Adenocarcinoma Cell;Affect;Animal Mod...,Howard C Crawford; Marina Pasca Di Magliano,Jeffrey Hildesheim,1497616,1497616,2017-09-25T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ANN ARBOR,MI,UNITED STATES
5151,1U01CA224146-01,U01CA224146,9449587,2017,Systematic interrogation of the pancreatic can...,Abstract Pancreatic ductal adenocarcinoma (PDA...,Aftercare;Atlases;Biology;Biopsy;Biopsy Specim...,William C Hahn,Steven F Nothwehr,1845039,1845039,2017-09-22T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,DANA-FARBER CANCER INST,BOSTON,MA,UNITED STATES
4081,1U01CA224146-01,U01CA224146,9449587,2017,Systematic interrogation of the pancreatic can...,Abstract Pancreatic ductal adenocarcinoma (PDA...,Aftercare;Atlases;Biology;Biopsy;Biopsy Specim...,William C Hahn,Steven F Nothwehr,1845039,1845039,2017-09-22T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,DANA-FARBER CANCER INST,BOSTON,MA,UNITED STATES
4082,1U01CA224175-01,U01CA224175,9450122,2017,Defining neoantigen immunodominance for antige...,PROJECT SUMMARY Checkpoint blockade immunother...,Address;Antigens;Biological Assay;Biological M...,Vinod P Balachandran; Steven D Leach,Sumana Mukherjee Dey,1850296,1850296,2017-09-25T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,SLOAN-KETTERING INST CAN RESEARCH,NEW YORK,NY,UNITED STATES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,5U01CA224348-03,U01CA224348,10251378,2021,Reprogramming PDAC tumor microenvironment to i...,We have recently shown in preclinical studies ...,Address;Affect;Angiotensin II;Angiotensin Rece...,Yves Boucher; Rakesh K. Jain,Peter Ujhazy,577095,577095,2021-09-10T12:09:00Z,2017-09-30T12:09:00Z,2022-08-31T12:08:00Z,RFA-CA-17-015,MASSACHUSETTS GENERAL HOSPITAL,BOSTON,MA,UNITED STATES
5167,5U24CA224020-03,U24CA224020,10254377,2021,Pancreatic Ductal Adenocarcinoma Translational...,ABSTRACT In contrast to the decline in mortali...,Administrative Coordination;Algorithms;Antigen...,Anirban Maitra; Subrata Sen,Peter Ujhazy,559976,559976,2021-08-12T12:08:00Z,2017-09-30T12:09:00Z,2022-08-31T12:08:00Z,RFA-CA-17-016,UNIVERSITY OF TX MD ANDERSON CAN CTR,HOUSTON,TX,UNITED STATES
4097,5U24CA224020-03,U24CA224020,10254377,2021,Pancreatic Ductal Adenocarcinoma Translational...,ABSTRACT In contrast to the decline in mortali...,Administrative Coordination;Algorithms;Antigen...,Anirban Maitra; Subrata Sen,Peter Ujhazy,559976,559976,2021-08-12T12:08:00Z,2017-09-30T12:09:00Z,2022-08-31T12:08:00Z,RFA-CA-17-016,UNIVERSITY OF TX MD ANDERSON CAN CTR,HOUSTON,TX,UNITED STATES
4098,6U01CA224193-04,U01CA224193,10602921,2021,Disrupting the immune and drug-privileged micr...,PROJECT SUMMARY Pancreatic ductal adenocarcino...,NaN,Sunil R Hingorani,Jeffrey Hildesheim,257096,257096,2022-05-27T12:05:00Z,2017-09-30T12:09:00Z,2022-08-31T12:08:00Z,RFA-CA-17-015,FRED HUTCHINSON CANCER CENTER,Seattle,WA,UNITED STATES


In [15]:
df2[df2.duplicated(keep=False)].sort_values(by='project_id')

,project_id,queried_project_id,application_id,fiscal_year,project_title,abstract_text,keywords,principal_investigators,program_officers,award_amount,nci_funded_amount,award_notice_date,project_start_date,project_end_date,opportunity_number,org_name,org_city,org_state,org_country
5150,1U01CA224145-01,U01CA224145,9449550,2017,Interrupting Cellular Crosstalk in the Immunos...,Abstract Immune therapy has shown great promis...,Ablation;Adenocarcinoma Cell;Affect;Animal Mod...,Howard C Crawford; Marina Pasca Di Magliano,Jeffrey Hildesheim,1497616,1497616,2017-09-25T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ANN ARBOR,MI,UNITED STATES
4080,1U01CA224145-01,U01CA224145,9449550,2017,Interrupting Cellular Crosstalk in the Immunos...,Abstract Immune therapy has shown great promis...,Ablation;Adenocarcinoma Cell;Affect;Animal Mod...,Howard C Crawford; Marina Pasca Di Magliano,Jeffrey Hildesheim,1497616,1497616,2017-09-25T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ANN ARBOR,MI,UNITED STATES
5151,1U01CA224146-01,U01CA224146,9449587,2017,Systematic interrogation of the pancreatic can...,Abstract Pancreatic ductal adenocarcinoma (PDA...,Aftercare;Atlases;Biology;Biopsy;Biopsy Specim...,William C Hahn,Steven F Nothwehr,1845039,1845039,2017-09-22T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,DANA-FARBER CANCER INST,BOSTON,MA,UNITED STATES
4081,1U01CA224146-01,U01CA224146,9449587,2017,Systematic interrogation of the pancreatic can...,Abstract Pancreatic ductal adenocarcinoma (PDA...,Aftercare;Atlases;Biology;Biopsy;Biopsy Specim...,William C Hahn,Steven F Nothwehr,1845039,1845039,2017-09-22T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,DANA-FARBER CANCER INST,BOSTON,MA,UNITED STATES
4082,1U01CA224175-01,U01CA224175,9450122,2017,Defining neoantigen immunodominance for antige...,PROJECT SUMMARY Checkpoint blockade immunother...,Address;Antigens;Biological Assay;Biological M...,Vinod P Balachandran; Steven D Leach,Sumana Mukherjee Dey,1850296,1850296,2017-09-25T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,SLOAN-KETTERING INST CAN RESEARCH,NEW YORK,NY,UNITED STATES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,5U01CA224348-03,U01CA224348,10251378,2021,Reprogramming PDAC tumor microenvironment to i...,We have recently shown in preclinical studies ...,Address;Affect;Angiotensin II;Angiotensin Rece...,Yves Boucher; Rakesh K. Jain,Peter Ujhazy,577095,577095,2021-09-10T12:09:00Z,2017-09-30T12:09:00Z,2022-08-31T12:08:00Z,RFA-CA-17-015,MASSACHUSETTS GENERAL HOSPITAL,BOSTON,MA,UNITED STATES
5167,5U24CA224020-03,U24CA224020,10254377,2021,Pancreatic Ductal Adenocarcinoma Translational...,ABSTRACT In contrast to the decline in mortali...,Administrative Coordination;Algorithms;Antigen...,Anirban Maitra; Subrata Sen,Peter Ujhazy,559976,559976,2021-08-12T12:08:00Z,2017-09-30T12:09:00Z,2022-08-31T12:08:00Z,RFA-CA-17-016,UNIVERSITY OF TX MD ANDERSON CAN CTR,HOUSTON,TX,UNITED STATES
4097,5U24CA224020-03,U24CA224020,10254377,2021,Pancreatic Ductal Adenocarcinoma Translational...,ABSTRACT In contrast to the decline in mortali...,Administrative Coordination;Algorithms;Antigen...,Anirban Maitra; Subrata Sen,Peter Ujhazy,559976,559976,2021-08-12T12:08:00Z,2017-09-30T12:09:00Z,2022-08-31T12:08:00Z,RFA-CA-17-016,UNIVERSITY OF TX MD ANDERSON CAN CTR,HOUSTON,TX,UNITED STATES
4098,6U01CA224193-04,U01CA224193,10602921,2021,Disrupting the immune and drug-privileged micr...,PROJECT SUMMARY Pancreatic ductal adenocarcino...,NaN,Sunil R Hingorani,Jeffrey Hildesheim,257096,257096,2022-05-27T12:05:00Z,2017-09-30T12:09:00Z,2022-08-31T12:08:00Z,RFA-CA-17-015,FRED HUTCHINSON CANCER CENTER,Seattle,WA,UNITED STATES


In [133]:
df3 = df.drop(columns='api_source_search')
df3

,project_id,queried_project_id,application_id,fiscal_year,project_title,abstract_text,keywords,principal_investigators,program_officers,award_amount,nci_funded_amount,award_notice_date,project_start_date,project_end_date,opportunity_number,org_name,org_city,org_state,org_country,program.program_id
0,1R01CA239701-01A1,R01CA239701,9995648,2020,Admixture analysis of acute lymphoblastic leuk...,Abstract Children with substantial African anc...,Accounting;Acute Lymphocytic Leukemia;Admixtur...,Michael E Scheurer; Logan G. Spector,Danielle L Daee,1898254,1898254,2020-07-23T12:07:00Z,2020-08-01T12:08:00Z,2023-07-31T12:07:00Z,PA-19-056,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...
1,3R01CA239701-01A1S1,R01CA239701,10307680,2021,Admixture analysis of acute lymphoblastic leuk...,This attachment is not required for the divers...,Acute Lymphocytic Leukemia;Admixture;African A...,Michael E Scheurer; Logan G. Spector,Danielle L Daee,11554,11554,2021-09-15T12:09:00Z,2021-08-01T12:08:00Z,2022-07-31T12:07:00Z,PA-21-071,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...
2,3R01CA239701-01A1S3,R01CA239701,10626271,2022,Admixture analysis of acute lymphoblastic leuk...,Abstract This application is being submitted i...,Acute Lymphocytic Leukemia;Administrative Supp...,Joseph Lubega; Michael E Scheurer; Logan G. Sp...,Danielle L Daee,197648,197648,2022-09-19T12:09:00Z,2020-08-01T12:08:00Z,2023-07-31T12:07:00Z,PA-20-272,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...
3,4R01CA239701-02,R01CA239701,10902170,2023,Admixture analysis of acute lymphoblastic leuk...,Modified Project Summary/Abstract Section Abst...,Acute Lymphocytic Leukemia;Admixture;African;A...,Michael E Scheurer; Logan G. Spector,Danielle L Daee,635018,635018,2023-09-15T12:09:00Z,2020-08-01T12:08:00Z,2025-07-31T12:07:00Z,PA-19-056,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES,ADMIRALStudyAdmixtureanalysisofacutelymphoblas...
4,1R21CA209848-01,R21CA209848,9185145,2016,Algorithms for Literature-Guided Multi-Platfor...,Project Abstract The development of accurate a...,Accounting;Address;Algorithms;Bayesian Modelin...,Dongjun Chung; Linda E Kelemen,David J Miller,195098,195098,2016-07-27T12:07:00Z,2016-08-01T12:08:00Z,2018-07-31T12:07:00Z,PAR-15-334,MEDICAL UNIVERSITY OF SOUTH CAROLINA,CHARLESTON,SC,UNITED STATES,ADVANCEDDEVELOPMENTOFINFORMATICSTECHNOLOGIESFO...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,5P50CA196530-04,P50CA196530,9529550,2018,Yale SPORE in Lung Cancer (YSILC): The Biology...,DESCRIPTION (provided by applicant): The Yale ...,American;Animal Disease Models;Animal Model;Be...,Roy S Herbst,Peter Ujhazy,2507598,2507598,2018-08-20T12:08:00Z,2015-08-26T12:08:00Z,2020-07-31T12:07:00Z,PAR-14-031,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES,YaleSPOREinLungCancerYSILCTheBiologyandPersona...
5996,5P50CA196530-05,P50CA196530,9767058,2019,Yale SPORE in Lung Cancer (YSILC): The Biology...,DESCRIPTION (provided by applicant): The Yale ...,American;Animal Disease Models;Animal Model;Be...,Roy S Herbst,Peter Ujhazy,2134771,2134771,2019-08-21T12:08:00Z,2015-08-26T12:08:00Z,2020-07-31T12:07:00Z,PAR-14-031,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES,YaleSPOREinLungCancerYSILCTheBiologyandPersona...
5997,5P50CA196530-07,P50CA196530,10203850,2021,Yale SPORE in Lung Cancer (YSILC): The Biology...,YALE SPORE IN LUNG CANCER (YSILC) OVERALL SUMM...,Address;Adoption;Animal Disease Models;Area;Aw...,Roy S Herbst,Peter Ujhazy,2003668,2003668,2021-09-01T12:09:00Z,2015-08-26T12:08:00Z,2025-07-31T12:07:00Z,PAR-18-313,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES,YaleSPOREinLungCancerYSILCTheBiologyandPersona...
5998,5P50CA196530-08,P50CA196530,10479786,2022,Yale SPORE in Lung Cancer (YSILC): The Biology...,YALE SPORE IN LUNG CANCER (YSILC) OVERALL SUMM...,Address;Adoption;Animal Disease Models;Are

In [84]:
def validate_duplicate_project_ids(df, project_id_col="project_id", program_id_col="program.program_id"):
    """
    Validates duplicate project IDs in a pandas DataFrame, ensuring that:
    - For rows with identical project_id values, program.program_id values are not identical.
    - Values in all other columns (excluding program.program_id) are identical.
    - Skips validation for project IDs that occur only once.

    Args:
        df (pd.DataFrame): The DataFrame to validate.
        project_id_col (str, optional): The name of the project ID column. Defaults to "project_id".
        program_id_col (str, optional): The name of the program ID column. Defaults to "program.program_id".

    Raises:
        ValueError: If any rows violate the validation conditions.
    """

    # Identify groups of rows with duplicate project IDs
    groups = df.groupby(project_id_col)

    for project_id, group_df in groups:
        if len(group_df) > 1:  # Only validate if there are duplicates
            # Check condition 1: program.program_id values must be different
            if len(group_df[program_id_col].unique()) <= 1:
                raise ValueError(f"Invalid duplicate project IDs found for project_id '{project_id}': program.program_id values are identical.")

            # Check condition 2: other columns must be identical
            other_cols = [col for col in group_df.columns if col not in [project_id_col, program_id_col]]
            if not group_df[other_cols].duplicated().all():
                raise ValueError(f"Invalid duplicate project IDs found for project_id '{project_id}': values in other columns are not identical.")


In [85]:
validate_duplicate_project_ids(df)

ValueError: Invalid duplicate project IDs found for project_id '1U01CA224145-01': values in other columns are not identical.

In [88]:
df[df['project_id'] =='1U01CA224145-01'].to_csv('test_duplicate.csv', index=False)

In [87]:
validate_duplicate_project_ids(df3)

ValueError: Invalid duplicate project IDs found for project_id '1U01CA224145-01': values in other columns are not identical.

In [92]:
test = df3[df3['project_id'] == '1U01CA224145-01']

In [94]:
test

,project_id,queried_project_id,application_id,fiscal_year,project_title,abstract_text,keywords,principal_investigators,program_officers,award_amount,nci_funded_amount,award_notice_date,project_start_date,project_end_date,opportunity_number,org_name,org_city,org_state,org_country,program.program_id
4080,1U01CA224145-01,U01CA224145,9449550,2017,Interrupting Cellular Crosstalk in the Immunos...,Abstract Immune therapy has shown great promis...,Ablation;Adenocarcinoma Cell;Affect;Animal Mod...,Howard C Crawford; Marina Pasca Di Magliano,Jeffrey Hildesheim,1497616,1497616,2017-09-25T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ANN ARBOR,MI,UNITED STATES,CONSORTIUMFORPANCREATICDUCTALADENOCARCINOMAPDA...
5150,1U01CA224145-01,U01CA224145,9449550,2017,Interrupting Cellular Crosstalk in the Immunos...,Abstract Immune therapy has shown great promis...,Ablation;Adenocarcinoma Cell;Affect;Animal Mod...,Howard C Crawford; Marina Pasca Di Magliano,Jeffrey Hildesheim,1497616,1497616,2017-09-25T12:09:00Z,2017-09-30T12:09:00Z,2020-08-31T12:08:00Z,RFA-CA-17-015,UNIVERSITY OF MICHIGAN AT ANN ARBOR,ANN ARBOR,MI,UNITED STATES,PancreaticCancerMicroenvironmentNetwork


In [117]:
# Select the two rows to compare
row1 = test.iloc[0]
row2 = test.iloc[1]


unequal_cols = np.where(row1 != row2)[0]
print(df.columns[unequal_cols])

Index(['org_country'], dtype='object')


In [95]:
row1['org_country']

'UNITED STATES'

In [96]:
row2['org_country']

'UNITED STATES'

In [97]:
row1['org_country'] == row2['org_country']

True

In [100]:
print(row1['org_country'])
print(row2['org_country'])
print(row1['org_country'] == row2['org_country'])

UNITED STATES
UNITED STATES
True


In [101]:
row1['org_country'] = row1['org_country'].strip()
row2['org_country'] = row2['org_country'].strip()
row1['org_country'] == row2['org_country']

C:\Users\dormanza\AppData\Local\Temp\1\ipykernel_6764\3067605293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row1['org_country'] = row1['org_country'].strip()
C:\Users\dormanza\AppData\Local\Temp\1\ipykernel_6764\3067605293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row2['org_country'] = row2['org_country'].strip()


True

In [102]:
row1.dtypes

dtype('O')

In [103]:
row2.detypes

AttributeError: 'Series' object has no attribute 'detypes'

In [110]:
type(row1['org_country'])

str

In [118]:
row1.drop('program.program_id',inplace=True)
row2.drop('program.program_id',inplace=True)

C:\Users\dormanza\AppData\Local\Temp\1\ipykernel_6764\101405260.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row1.drop('program.program_id',inplace=True)
C:\Users\dormanza\AppData\Local\Temp\1\ipykernel_6764\101405260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row2.drop('program.program_id',inplace=True)


In [119]:
row1 == row2

project_id                 True
queried_project_id         True
application_id             True
fiscal_year                True
project_title              True
abstract_text              True
keywords                   True
principal_investigators    True
program_officers           True
award_amount               True
nci_funded_amount          True
award_notice_date          True
project_start_date         True
project_end_date           True
opportunity_number         True
org_name                   True
org_city                   True
org_state                  True
org_country                True
dtype: bool

In [120]:
unequal_cols = np.where(row1 != row2)[0]
print(df.columns[unequal_cols])

Index([], dtype='object')


In [121]:
def validate_duplicate_project_ids(df, project_id_col="project_id", program_id_col="program.program_id"):
    """
    Validates duplicate project IDs in a pandas DataFrame according to specified conditions.

    Args:
        df (pd.DataFrame): The DataFrame to validate.
        project_id_col (str, optional): The name of the project ID column. Defaults to "project_id".
        program_id_col (str, optional): The name of the program ID column. Defaults to "program.program_id".

    Raises:
        ValueError: If any rows violate the validation conditions.
    """

    # Identify groups of rows with duplicate project IDs
    groups = df.groupby(project_id_col)

    for project_id, group_df in groups:
        if len(group_df) == 1:  # Skip if project_id occurs only once
            continue

        # Check if program.program_id values are different
        if len(group_df[program_id_col].unique()) <= 1:
            raise ValueError(f"Invalid duplicate project IDs found for project_id '{project_id}': program.program_id values are identical.")

        # Check if other columns are identical (excluding program.program_id)
        other_cols = [col for col in group_df.columns if col not in [project_id_col, program_id_col]]
        if not group_df[other_cols].duplicated().all():
            # Identify mismatched columns and values
            mismatch_cols = other_cols[~group_df[other_cols].drop_duplicates().equals(group_df[other_cols].iloc[0])]
            mismatch_values = group_df[mismatch_cols].values.tolist()
            raise ValueError(
                f"Invalid duplicate project IDs found for project_id '{project_id}': values in columns {mismatch_cols} are not identical. Mismatched values:\n{mismatch_values}"
            )

In [122]:
print(df.columns)
print(df2.columns)
print(df3.columns)

Index(['project_id', 'queried_project_id', 'application_id', 'fiscal_year',
       'project_title', 'abstract_text', 'keywords', 'principal_investigators',
       'program_officers', 'award_amount', 'nci_funded_amount',
       'award_notice_date', 'project_start_date', 'project_end_date',
       'opportunity_number', 'api_source_search', 'org_name', 'org_city',
       'org_state', 'org_country', 'program.program_id'],
      dtype='object')
Index(['project_id', 'queried_project_id', 'application_id', 'fiscal_year',
       'project_title', 'abstract_text', 'keywords', 'principal_investigators',
       'program_officers', 'award_amount', 'nci_funded_amount',
       'award_notice_date', 'project_start_date', 'project_end_date',
       'opportunity_number', 'org_name', 'org_city', 'org_state',
       'org_country'],
      dtype='object')
Index(['project_id', 'queried_project_id', 'application_id', 'fiscal_year',
       'project_title', 'abstract_text', 'keywords', 'principal_investigators',

In [123]:
validate_duplicate_project_ids(df)

ValueError: Invalid duplicate project IDs found for project_id '1U01CA224145-01': values in columns org_country are not identical. Mismatched values:
['UNITED STATES', 'UNITED STATES']

In [125]:
validate_duplicate_project_ids(df3)

ValueError: Invalid duplicate project IDs found for project_id '1U01CA224145-01': values in columns org_country are not identical. Mismatched values:
['UNITED STATES', 'UNITED STATES']

In [126]:
df3.dtypes

project_id                 object
queried_project_id         object
application_id              int64
fiscal_year                 int64
project_title              object
abstract_text              object
keywords                   object
principal_investigators    object
program_officers           object
award_amount                int64
nci_funded_amount           int64
award_notice_date          object
project_start_date         object
project_end_date           object
opportunity_number         object
org_name                   object
org_city                   object
org_state                  object
org_country                object
program.program_id         object
dtype: object

In [127]:
df3['org_country'] = df3['org_country'].str.strip()
validate_duplicate_project_ids(df3)

ValueError: Invalid duplicate project IDs found for project_id '1U01CA224145-01': values in columns org_country are not identical. Mismatched values:
['UNITED STATES', 'UNITED STATES']

In [129]:
df3['org_country'] = df3['org_country'].str.normalize('NFKC')

In [134]:
validate_duplicate_project_ids(df3)

ValueError: Invalid duplicate project IDs found for project_id '1U01CA224145-01': values in columns org_country are not identical. Mismatched values:
['UNITED STATES', 'UNITED STATES']

In [132]:
print(df3['org_country'].str.encode('utf-8').unique())

[b'UNITED STATES' b'CANADA' b'GERMANY' b'FRANCE' b'SWEDEN' b'SOUTH AFRICA'
 b'AUSTRALIA' b'UGANDA' b'RWANDA' b'KENYA' b'UNITED KINGDOM']


In [135]:
df3['org_country'] = df3['org_country'].str.normalize('NFC')

In [136]:
validate_duplicate_project_ids(df3)

ValueError: Invalid duplicate project IDs found for project_id '1U01CA224145-01': values in columns org_country are not identical. Mismatched values:
['UNITED STATES', 'UNITED STATES']

In [138]:
groups = df3.groupby('project_id')

for project_id, group_df in groups:
    if len(group_df) > 1:
        print(project_id)

1U01CA224145-01
1U01CA224146-01
1U01CA224175-01
1U01CA224193-01
1U01CA224348-01
1U24CA224020-01
2U24CA224285-02
2U24CA224309-02
2U24CA224319-02
2U24CA224331-02
3P50CA098131-20S1
3P50CA098131-21S1
3P50CA100632-20S1
3P50CA171963-10S1
3P50CA236733-05S1
3R01CA239701-01A1S3
3U10CA180821-08S1
3U10CA180821-08S2
3U10CA180821-09S1
3U10CA180821-09S2
3U10CA180821-09S5
3U10CA180821-10S1
3U10CA180821-10S2
4U01CA224145-02
4U01CA224146-02
4U01CA224175-02
4U01CA224193-02
4U01CA224348-02
4U24CA224020-02
5U01CA224145-03
5U01CA224146-03
5U01CA224175-03
5U01CA224193-03
5U01CA224348-03
5U24CA224020-03
6U01CA224193-04


In [143]:
# def validate_duplicate_project_ids(df, project_id_col="project_id", program_id_col="program.program_id"):
#     """
#     Validates duplicate project IDs in a pandas DataFrame according to specified conditions.

#     Args:
#         df (pd.DataFrame): The DataFrame to validate.
#         project_id_col (str, optional): The name of the project ID column. Defaults to "project_id".
#         program_id_col (str, optional): The name of the program ID column. Defaults to "program.program_id".

#     Raises:
#         ValueError: If any rows violate the validation conditions.
#     """

#     # Identify groups of rows with duplicate project IDs
#     groups = df.groupby(project_id_col)

#     for project_id, group_df in groups:
#         if len(group_df) == 1:  # Skip if project_id occurs only once
#             continue

#         # Check if program.program_id values are different
#         if len(group_df[program_id_col].unique()) <= 1:
#             raise ValueError(f"Invalid duplicate project IDs found for project_id '{project_id}': program.program_id values are identical.")

#         # Explicitly drop program.program_id column and check remaining values
#         remaining_cols = group_df.drop(program_id_col, axis=1)
#         if not remaining_cols.duplicated().all():
#             # Identify mismatched columns and values
#             mismatch_cols = remaining_cols.columns[~remaining_cols.drop_duplicates().equals(remaining_cols.iloc[0])]
#             mismatch_values = remaining_cols[mismatch_cols].values.tolist()
#             raise ValueError(
#                 f"Invalid duplicate project IDs found for project_id '{project_id}': values in columns {mismatch_cols} are not identical. Mismatched values:\n{mismatch_values}")

## Day 2: Try a different approach

In [4]:
test_df = pd.DataFrame({'project_id': ['project_A', 'project_A', 'project_B', 'project_B', 'project_C', 'project_C', 'project_D', 'project_F', 'project_F'],
                   'other_col_01': [123, 123, 123, 456, 123, 123, 123, 789, 789],
                   'other_col_02': ['abc', 'abc', 'abc', 'def', 'abc', 'abc', 'abc', 'xyz', 'xyz'],
                   'program.program_id': ['program_A', 'program_B', 'program_C', 'program_D', 'program_E', 'program_E', 'program_A', 'program_X', 'program_X']})

In [116]:
test_df

,project_id,other_col_01,other_col_02,program.program_id
0,project_A,123,abc,program_A
1,project_A,123,abc,program_B
2,project_B,123,abc,program_C
3,project_B,456,def,program_D
4,project_C,123,abc,program_E
5,project_C,123,abc,program_E
6,project_D,123,abc,program_A
7,project_F,789,xyz,program_X
8,project_F,789,xyz,program_X


In [117]:
df = test_df.copy()
node_id = 'project_id'
link_id = 'program.program_id'

In [118]:
# Get only rows with duplicated node_ids 
duplicated_nodes = df[df.duplicated(subset=node_id, keep=False)]
duplicated_nodes

,project_id,other_col_01,other_col_02,program.program_id
0,project_A,123,abc,program_A
1,project_A,123,abc,program_B
2,project_B,123,abc,program_C
3,project_B,456,def,program_D
4,project_C,123,abc,program_E
5,project_C,123,abc,program_E
7,project_F,789,xyz,program_X
8,project_F,789,xyz,program_X


In [119]:
# Get true duplicates. One should be removed for production. Both should be removed for next validation
true_duplicates = duplicated_nodes[duplicated_nodes.duplicated(keep=False)
                                   ].assign(error_reason=f"True duplicate including link_id. Kept first row for production.")
true_duplicates 

,project_id,other_col_01,other_col_02,program.program_id,error_reason
4,project_C,123,abc,program_E,True duplicate including link_id. Kept first r...
5,project_C,123,abc,program_E,True duplicate including link_id. Kept first r...
7,project_F,789,xyz,program_X,True duplicate including link_id. Kept first r...
8,project_F,789,xyz,program_X,True duplicate including link_id. Kept first r...


In [121]:
# Empty dataframe to store mismatch rows
mismatch_df = pd.DataFrame()

# Assuming df is your DataFrame, and node_id is the column for grouping
for node, group_df in duplicated_nodes.groupby(node_id):
    
    # check for mismatched values in other columns within the same node group
    value_mismatches = group_df[~group_df.duplicated(subset=['other_col_01', 'other_col_02'], keep=False)
                                ].assign(error_reason=f"Mismatched values in columns with same node_id. Both dropped for production.")

    mismatch_df = pd.concat([mismatch_df, value_mismatches])

display(mismatch_df)

,project_id,other_col_01,other_col_02,program.program_id,error_reason
2,project_B,123,abc,program_C,Mismatched values in columns with same node_id...
3,project_B,456,def,program_D,Mismatched values in columns with same node_id...


In [122]:
# Combine list of annotated invalid duplicates for export to csv
invalid_dupllicates_df = pd.concat([true_duplicates, mismatch_df])
invalid_dupllicates_df

,project_id,other_col_01,other_col_02,program.program_id,error_reason
4,project_C,123,abc,program_E,True duplicate including link_id. Kept first r...
5,project_C,123,abc,program_E,True duplicate including link_id. Kept first r...
7,project_F,789,xyz,program_X,True duplicate including link_id. Kept first r...
8,project_F,789,xyz,program_X,True duplicate including link_id. Kept first r...
2,project_B,123,abc,program_C,Mismatched values in columns with same node_id...
3,project_B,456,def,program_D,Mismatched values in columns with same node_id...


In [7]:
def validate_unique_nodes(df, column_configs, datatype):
    """
    Validate that all node_ids are unique or valid duplicates. 
    
    For many-to-many nodes, validate that any duplicates only differ in the 
    link_id column. 

    Args:
        df (pd.DataFrame): Input DataFrame.
        node_id (str, optional): Name of the column containing node identifiers. 
            Defaults to "project_id".
        link_id (str, optional): Name of the column containing link identifiers. 
            Defaults to "program.program_id".

    Returns:
        pd.DataFrame: A new DataFrame with invalid duplicate rows removed.

    Raises:
        ValueError: If the input DataFrame has no specified node_id column.

    """
    # Get node_id and link_id from column configurations
    node_id = column_configs[datatype].get('node_id')
    link_id = column_configs[datatype].get('link_id', None)
    
    # Check if the specified node_id column exists
    if node_id not in df.columns:
        raise ValueError(f"Node ID column '{node_id}' not found in DataFrame.")

    # Get only rows with duplicated node_ids 
    dup_nodes = df[df.duplicated(subset=node_id, keep=False)]
    
    # Get true duplicates where values in all columns are identical
    true_duplicates = dup_nodes[dup_nodes.duplicated(keep=False)].assign(
        error_reason=f"True duplicate in all columns. "
                     f"Kept first row for production.")
    
    # Empty dataframe to store mismatch rows
    mismatch_df = pd.DataFrame()

    # Get list of value columns to check for mismatch
    if link_id is None:
        value_cols = list(set(df.columns) - set(node_id))
    else: 
        value_cols = list(set(df.columns) - set([node_id, link_id]))

    # Group rows by node_id for shared value checking
    for node, node_df in dup_nodes.groupby(node_id):
        
        # Check for mismatched values in other columns within the same node
        value_mismatches = node_df[~node_df.duplicated(
            subset=value_cols, keep=False)
            ].assign(error_reason=f"Mismatched values in columns with same "
                                  f"node_id. Both dropped for production.")

        # Add to running list of rows with mismatched values
        mismatch_df = pd.concat([mismatch_df, value_mismatches])

    # Combine list of annotated invalid duplicates for export to csv
    invalid_dupllicates_df = pd.concat([true_duplicates, mismatch_df])
    invalid_duplicates_path = 'invalid_duplicates.csv'
    invalid_dupllicates_df.to_csv(invalid_duplicates_path, index=False)

    # Return a cleaned df with only unique node_ids or valid duplicates
    # Remove all mismatched node_ids and any true duplicates after the first
    valid_df = df[~df[node_id].isin(mismatch_df[node_id]) 
                  & ~df.duplicated(subset=node_id, keep='first')
                  ].reset_index(drop=True)

    return valid_df

In [8]:
validate_unique_nodes(test_df, config.COLUMN_CONFIGS, datatype="grant")

,project_id,other_col_01,other_col_02,program.program_id
0,project_A,123,abc,program_A
1,project_C,123,abc,program_E
2,project_D,123,abc,program_A
3,project_F,789,xyz,program_X
